In [38]:
import yfinance as yf
import talib
import pandas as pd
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide

# data preprocess
target_stock = "2603"
threshold = 0.01
stock_df = pd.read_csv(r'C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\data\2603 2023.csv', encoding='utf-8', encoding_errors= 'ignore')
five_day_min_low = talib.MIN(stock_df['Low'], timeperiod=5) # 五天最低價當作long strategy的停損點
five_day_max_high = talib.MAX(stock_df['High'], timeperiod=5) # 五天最高價當作short strategy的停損點
stock_df['five_day_min_low'] = five_day_min_low
stock_df['five_day_max_high'] = five_day_max_high
stock_df = stock_df[25:-1]
predicted_price_df = pd.read_csv(r"C:\Users\Admin\Desktop\量化交易\lstm 預測股價(squeeze中包含的技術分析)\LSTM_stock_predicting\predicted_result\ResGRU 2603 2023 predicted prices.csv", encoding='utf-8', encoding_errors = 'ignore')
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df.index = stock_df['Date']
stock_df['predicted_price'] = predicted_price_df['predicted_price'].values
stock_df['true_price'] = predicted_price_df['true_price'].values

# long short condition
stock_df['predicted_price_up'] = (stock_df['predicted_price'] > stock_df['predicted_price'].shift(1)).astype(int)
stock_df['actual_price_up'] = (stock_df['Close'] > stock_df['Close'].shift(1)).astype(int)
stock_df['long_condition'] = (stock_df['true_price'].shift(1)<=stock_df['predicted_price']) & ((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1) >= threshold)
stock_df['short_condition'] = (stock_df['true_price'].shift(1)>stock_df['predicted_price']) & (abs((stock_df['predicted_price']-stock_df['true_price'].shift(1))/stock_df['true_price'].shift(1)) >= threshold)
stock_df['long_condition'] = stock_df['long_condition'].astype(int)
stock_df['short_condition'] = stock_df['short_condition'].astype(int)

stock_df

,Date,Capacity,Turnover,Open,High,Low,Close,Change,Transcation,buy_Dealer_Hedging,...,sell_Foreign_Investor,sell_Investment_Trust,five_day_min_low,five_day_max_high,predicted_price,true_price,predicted_price_up,actual_price_up,long_condition,short_condition
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-17,2023-02-17,10357721,1514125553,146.5,147.0,145.5,146.0,-2.0,7528,68000,...,4801322,23643,142.5,149.0,0.648761,0.637041,0,0,0,0
2023-02-20,2023-02-20,8951220,1312724748,146.0,147.5,145.0,147.0,1.0,5729,348037,...,2118000,12000,145.0,149.0,0.646760,0.641694,0,1,1,0
2023-02-21,2023-02-21,42687050,6486525128,148.5,155.0,147.0,155.0,8.0,22782,1193000,...,3415406,1000,145.0,155.0,0.646459,0.678920,0,1,0,0
2023-02-22,2023-02-22,35721736,5509512316,154.0,156.0,152.0,152.5,-2.5,17617,271000,...,9044276,7642,145.0,156.0,0.647247,0.667287,1,0,0,1
2023-02-23,2023-02-23,34603881,5423905167,153.5,158.5,153.5,158.0,5.5,19249,861222,...,3835200,5000,145.0,158.5,0.650299,0.692880,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,2023-12-22,60997658,9257968913,152.5,153.5,149.5,153.0,1.0,29123,488699,...,18284859,640000,136.0,153.5,0.606464,0.669614,1,1,0,1
2023-12-25,2023-12-25,89473186,12839164994,146.0,147.5,140.5,142.0,-11.0,45151,452023,...,24598528,2100000,140.5,153.5,0.619511,0.618427,1,0,0,1
2023-12-26,2023-12-26,39593610,5760984388,142.5,147.5,142.0,146.5,4.5,20847,514500,...,11837391,15000,140.5,153.5,0.625039,0.639367,1,1,1,0


strategy


In [39]:
# long strategy
signals_long = np.zeros(len(stock_df))
partition_size_long = 0
stop_loss_long = 0
for i in range(len(stock_df)):
    if partition_size_long == 0:
        if stock_df['long_condition'][i] == 1:
            signals_long[i] = 1
            partition_size_long = 1
            stop_loss_long = stock_df['five_day_min_low'][i]
    if partition_size_long == 1:
        if (stock_df['Close'][i] <= stop_loss_long) or (stock_df['short_condition'][i] == 1): # 停利停損
            signals_long[i] = -1
            stop_loss_long = 0 # 停損點重置
            partition_size_long = 0 # 平倉

# short strategy
signals_short = np.zeros(len(stock_df))
partition_size_short = 0
stop_loss_short = 0
for i in range(len(stock_df)):
    if partition_size_short == 0:
        if stock_df['short_condition'][i] == 1:
            signals_short[i] = -1
            partition_size_short = -1
            stop_loss_short = stock_df['five_day_max_high'][i]
    if partition_size_short == -1:
        if (stock_df['Close'][i] >= stop_loss_short) or (stock_df['long_condition'][i] == 1):# 停利停損
            signals_short[i] = 1
            stop_loss_short = 0 # 停損點重置
            partition_size_short = 0 # 平倉

entries_long = signals_long == 1
exits_long = signals_long == -1
entries_short = signals_short == -1
exits_short = signals_short == 1
pf = vbt.Portfolio.from_signals(stock_df['Open'], 
                                    entries=entries_long,
                                    exits=exits_long, 
                                    short_entries=entries_short,
                                    short_exits=exits_short,
                                    fees = 0,
                                    freq='1D',
                                    direction='both',)
# print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
print(pf.stats().to_string())
'''entries_short = signals_short == -1
exits_short = signals_short == 1
pf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')'''
# print(pf_short.stats().to_string())

Start                               2023-02-17 00:00:00
End                                 2023-12-28 00:00:00
Period                                213 days 00:00:00
Start Value                                       100.0
End Value                                    270.373427
Total Return [%]                             170.373427
Benchmark Return [%]                           1.023891
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              12.741264
Max Drawdown Duration                  33 days 00:00:00
Total Trades                                         39
Total Closed Trades                                  38
Total Open Trades                                     1
Open Trade PnL                                  1.80249
Win Rate [%]                                  60.526316
Best Trade [%]                                44.638554
Worst Trade [%]                               -6

C:\Users\Admin\AppData\Local\Temp\ipykernel_12520\2404176544.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_12520\2404176544.py:10: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_12520\2404176544.py:12: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_12520\2404176544.py:23: FutureWarning:

Series.__ge

"entries_short = signals_short == -1\nexits_short = signals_short == 1\npf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')"

In [40]:
pf.plot().show()